In [2]:
# Libraries

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import re

In [3]:
# define classes of information to be collected

class Reviews:
    def __init__(self, review_id, item_id, review_time, review_variant, star_rating, review_content):
        self.review_id = review_id
        self.item_id=item_id
        self.review_time = review_time
        self.review_variant = review_variant
        self.star_rating  = star_rating
        self.review_content = review_content

class Items:
    def __init__(self, item_id, item_name, item_ratings, ratings_num, item_sold, shop_name, item_price, item_url, item_desc, image_url):
        self.item_id=item_id
        self.item_name=item_name
        self.item_ratings=item_ratings
        self.ratings_num=ratings_num
        self.item_sold=item_sold
        self.shop_name=shop_name
        self.item_price=item_price
        self.item_url=item_url
        self.item_desc=item_desc
        self.image_url=image_url


In [4]:
#Create Chrome options
chrome_options= Options()
chrome_options.add_argument('--no sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('disable-inforbars')

# Set the path to the Chrome driver executable
chrome_driver_path = 'D:\Chrome web driver\chromedriver-win64\chromedriver.exe'

# Create an instance of the Chrome browser
browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

# Navigate to a URL 
browser.get("https://shopee.com.my/")

C:\Users\Asus\AppData\Local\Temp\ipykernel_5436\1499064468.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)


In [79]:
soup=BeautifulSoup(browser.page_source, "html.parser")

the_links=[]

parent_div = soup.findAll('div', class_="col-xs-2-4 shopee-search-item-result__item")

for parent_div in parent_div:
    a=parent_div.find('a')
    if (a!=None):
        href=a.get('href')
        the_links.append(href)

        # print the links
        print(href)
        

/TITISU🧻-80pcs-Shoe-Cleaning-Wet-Tissue-Quick-Wipes-Sneakers-Care-Shoes-cleaning-Solution-i.72251243.13444757171?sp_atk=592bff36-1c2e-494e-bd64-501948859c6f&xptdk=592bff36-1c2e-494e-bd64-501948859c6f
/DC-Home-Men-Buttery-Soft-Comfort-Sandals-DH-HA-3003-i.134516310.22104533350?sp_atk=bd76a619-5a92-4c9c-aa0c-c17a84cff210&xptdk=bd76a619-5a92-4c9c-aa0c-c17a84cff210
/(Size-35-44)READY-STOCK🔥Kasut-Lelaki🔥HIGH-Top-camouflage-Sneakers-kasut-perempuan-New-Canvas-Shoes-i.549089643.12877933687?sp_atk=8fe6f32d-37fd-484b-855d-ba2e744ad59f&xptdk=8fe6f32d-37fd-484b-855d-ba2e744ad59f
/AMOS-Fashion-Safety-Shoes-Kasut-Keselamatan-Heavy-Duty-Saftyshoes-Safety-Boots-i.273896127.17525806208?sp_atk=b257074d-1d94-40d6-bbd7-240041e4f614&xptdk=b257074d-1d94-40d6-bbd7-240041e4f614
/READY-STOCK-SNEAKERS-CLEANING-TOOLS-SHOES-CLEANING-COMBO-SET-SHOES-CARE-AND-CLEANING-TOOLS-i.74589099.4351883855?sp_atk=efd9fce0-1efc-428b-a5e7-16769517e234&xptdk=efd9fce0-1efc-428b-a5e7-16769517e234
/READY-STOCK⭐-TWICELOVE-Men's-Sum

In [80]:
len(the_links)

60

In [81]:
items_list=[]
reviews_list=[]

In [82]:
for i in range (0,len(the_links)):
    
    # open link
    links='https://shopee.com.my'+ the_links[i]
    browser.get(links)
    time.sleep(10)

    # parse html using beautifulsoup
    soup=BeautifulSoup(browser.page_source, "html.parser")

    # extract product information
    id=uuid.uuid4()
    title = soup.find('div', attrs={'class':'_44qnta'}).find('span').get_text(strip=True)
    ratings=soup.find('div', class_="_1k47d8 _046PXf").get_text(strip=True)
    ratings_num=soup.select('div._1k47d8:not(div._1k47d8._046PXf)')[0].get_text(strip=True)
    sold=soup.find('div', class_="e9sAa2").get_text(strip=True)
    shop_name=soup.find('div',class_="VlDReK").get_text(strip=True)
    price = soup.find('div', class_='pqTWkA').get_text(strip=True)

    # extract description
    descriptions = soup.find_all('p', class_='irIKAp')
    

    desc_array=[]
    for description in descriptions:
        desc = re.sub(r'<[^>]+>', '', str(description))  # Remove HTML tags
        desc = desc.strip()  # Remove leading/trailing whitespaces
        desc_array.append(desc)

    text = "\n".join(desc_array)

    # extract picture url
    img_tag= soup.find('img', class_='_7D4JtJ')

    if img_tag:
        image = img_tag['src']
    else:
        image=None

    # append item to a list
    items_list.append(Items(id,title,ratings,ratings_num,sold,shop_name, price, links, description, image))

    # reviews extraction
    elements = browser.find_elements(By.CLASS_NAME,"product-rating-overview__filter")
    counter=5
    for j in range(1,6):
        # select star categories
        elements[j].click()
        time.sleep(2)

        # loop is for each category
        for k in range (0,31): 
            soup=BeautifulSoup(browser.page_source, "html.parser")
            review_containers=soup.findAll('div',class_="shopee-product-rating__main")

            no_reviews_found = False

            for rc in review_containers:
                review_id=uuid.uuid4()
                item_id=id
                time_var=rc.find('div',class_="shopee-product-rating__time").get_text(strip=True).split("|")

                if(len(time_var)==2):
                    review_time=time_var[0]
                    variant=time_var[1]
                else:
                    review_time=time_var[0]
                    variant="None"

                star_rating=counter
                reviews=rc.find('div',class_="Rk6V+3")

                if(reviews!=None):
                    reviews=reviews.get_text(strip=True)
                else:
                    reviews="None"
                    no_reviews_found = True

                reviews_list.append(Reviews(review_id, item_id, review_time,variant,star_rating,reviews))
            
            # no more reviews break
            if no_reviews_found:
                print("No reviews found, breaking the loop")
                break

            time.sleep(1)
            try:
                WebDriverWait(browser, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class^='shopee-icon-button shopee-icon-button--right "))).click()
            except:
                print("not found")
                break
            time.sleep(1)
        
        counter=counter-1
        time.sleep(2)
    print(i) 
    
    

No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
0
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
1
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
2
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
3
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
4
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, breaking the loop
No reviews found, 

In [83]:
items_df = pd.DataFrame([vars(obj) for obj in items_list])
reviews_df = pd.DataFrame([vars(obj) for obj in reviews_list])

In [84]:
items_df.shape

(60, 10)

In [85]:
items_df.head()

,item_id,item_name,item_ratings,ratings_num,item_sold,shop_name,item_price,item_url,item_desc,image_url
0,ad9c23a8-e31b-4769-a302-d12eff6465fc,TITISU🧻 [80pcs] Shoe Cleaning Wet Tissue Quick...,4.9,7.3k,46.8k,ATP Professional Business Solution,RM0.50 - RM1.60,https://shopee.com.my/TITISU🧻-80pcs-Shoe-Clean...,[➡️ 15 x 20cm],https://down-my.img.susercontent.com/file/a024...
1,1dc79e34-5243-402b-a278-78a21676d8e3,DC Home Men Buttery Soft Comfort Sandals DH-HA...,4.9,1.9k,4.3k,Dr Cardin,RM39.00,https://shopee.com.my/DC-Home-Men-Buttery-Soft...,[- Sole Thickness: 40 mm],https://down-my.img.susercontent.com/file/4a3e...
2,307b4965-d15b-45a7-9d41-f87b2fb5c555,(Size 35-44)READY STOCK🔥Kasut Lelaki🔥HIGH Top ...,4.8,321,747,Ouyuyan Official Store,RM16.38 - RM16.77,https://shopee.com.my/(Size-35-44)READY-STOCK🔥...,[Applicable People: Youth (18-40 Years Old)],https://down-my.img.susercontent.com/file/e80f...
3,5445a22d-df11-412b-bf46-56dc29a56c04,AMOS Fashion Safety Shoes Kasut Keselamatan He...,4.9,898,2k,BellaChan Wholesale,RM42.45,https://shopee.com.my/AMOS-Fashion-Safety-Shoe...,[READY STOCK\n✔DROPSHIP | RESELLER | WHOLESALE...,https://down-my.img.susercontent.com/file/8263...
4,d9e50440-d45e-476c-bf9a-b6c974ec4344,READY STOCK SNEAKERS CLEANING TOOLS SHOES CLEA...,4.9,1.8k,3.9k,Smart Connection,RM4.90 - RM26.50,https://shopee.com.my/READY-STOCK-SNEAKERS-CLE...,[SNEAKERS CLEANING TOOLS SHOES CLEANING COMBO ...,https://down-my.img.susercontent.com/file/571b...


In [86]:
reviews_df.shape

(7575, 6)

In [87]:
reviews_df.tail()

,review_id,item_id,review_time,review_variant,star_rating,review_content
7570,0fbc4117-7c37-4d6b-bcdf-54f1b8ad91b9,9e63142c-c93b-4d63-bb04-baf312cdaaee,2023-01-30 11:11,"Variation: BROWN LOW-CUT,42",1,Comfortability:okMaterial:okDelivery dan custo...
7571,d8e2ffc4-275c-45dc-94c9-4fa214d7cade,9e63142c-c93b-4d63-bb04-baf312cdaaee,2023-02-22 23:31,"Variation: BLACK HI-CUT,44",1,Comfortability:sangt terukMaterial:xsesuai pak...
7572,03c272f4-3111-45d9-9869-f7a561095fdd,9e63142c-c93b-4d63-bb04-baf312cdaaee,2023-09-12 08:33,"Variation: BROWN LOW-CUT,47",1,Kaki size 47. Tapi x muat. Kecik dari size
7573,d7afada3-d1c2-428c-b990-9ac45ab047f8,9e63142c-c93b-4d63-bb04-baf312cdaaee,2023-09-07 10:25,"Variation: BROWN HI-CUT,44",1,None
7574,2b33e31d-06a4-47f3-9a2a-7f3906ac4407,9e63142c-c93b-4d63-bb04-baf312cdaaee,2023-05-04 12:43,"Variation: BLACK HI-CUT,42",1,None


In [78]:
reviews_df.to_json('menshoes_reviews_1.json', orient='records', lines=True, force_ascii=False, default_handler=str)
items_df.to_json('menshoes_items_1.json', orient='records', lines=True, force_ascii=False, default_handler=str)